<a href="https://colab.research.google.com/github/anand-verma/UPSC-CSE/blob/main/Add_Bookmark_to_Onenote_exported_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fitz
!pip install PyMuPDF

In [ ]:
import fitz  # PyMuPDF
import re
from IPython.display import display, HTML

# Function to extract titles and date-time stamps
def extract_titles_with_timestamps(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        titles_with_pages = []
        date_time_regex = re.compile(r'\d{2} \w+ \d{4}')
        time_regex = re.compile(r'\d{2}:\d{2}')
        default_title = "Untitled Page"

        for page_num in range(len(doc)):
            try:
                page = doc.load_page(page_num)
                text = page.get_text("text")
                lines = text.split('\n')

                # Check only the last 8 lines
                lines_to_check = lines[-8:] if len(lines) > 8 else lines

                title = default_title
                date_found = False
                time_found = False

                # Check from the last line in reverse
                for i in range(len(lines_to_check) - 1, -1, -1):
                    line = lines_to_check[i].strip()
                    #print (str(i) +" line is "+ line)
                    if time_regex.match(line):
                        time_found = True
                    elif date_time_regex.match(line):
                        date_found = True
                    elif date_found and time_found:
                        title = lines_to_check[i].strip() if i < len(lines_to_check) else default_title
                        titles_with_pages.append((title if title else default_title, page_num+1))
                        break  # Break the loop after finding the title

            except Exception as e:
                print(f"Error processing page {page_num}: {e}")

        return titles_with_pages

    except Exception as e:
        print(f"Error opening or processing the PDF: {e}")
        return []

In [ ]:
# Add bookmarks to the PDF
def add_bookmarks_to_pdf(pdf_path, bookmarks):
    try:
        doc = fitz.open(pdf_path)
        toc = []
        for title, page_num in bookmarks:
            toc.append((1, title, page_num))

        # Set the table of contents (TOC)
        doc.set_toc(toc)
        output_path = "/content/output_with_bookmarks.pdf"
        doc.save(output_path)
        return output_path

    except Exception as e:
        print(f"Error adding bookmarks to the PDF: {e}")
        return None

In [ ]:
from google.colab import files


# Upload file
uploaded = files.upload()

for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')
    pdf_path = filename


In [ ]:
# Extract titles and timestamps
titles_with_pages = extract_titles_with_timestamps(pdf_path)
print(titles_with_pages)

# Add bookmarks to the PDF
output_pdf_path = add_bookmarks_to_pdf(pdf_path, titles_with_pages)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Provide download link if the output file is successfully created
if output_pdf_path:
    files.download(output_pdf_path)
else:
    print("Failed to create the PDF with bookmarks.")